In [22]:
from tensorflow import keras
import numpy as np
from sklearn.datasets import load_iris
from Activation import Activation

class Network:
    
    def __init__(self):
        self.W = {
            'W1':np.random.randn(4,100),
            'W2':np.random.randn(100,50),
            'W3':np.random.randn(50,3)
        }
        self.b = {
            'b1':np.random.randn(100),
            'b2':np.random.randn(50),
            'b3':np.random.randn(3)
        }
        self.Activation ={
            'sigmoid':Activation.sigmoid,
            'relu':Activation.relu,
            'softmax':Activation.softmax
        }
        
    
    def predict(self,x):
        result = np.dot(x,self.W['W1']) + self.b['b1']
        for i in range(len(self.W)-2):
            result = np.dot(result,self.W['W'+str(i+2)]) + self.b['b'+str(i+2)]
            result = Activation.relu(result)    
        result = np.dot(result,self.W['W'+str(len(self.W))]) + self.b['b'+str(len(self.W))]    
        return Activation.softmax(result)
    
    def loss(self,x,t):
        self.y = self.predict(x)
        loss = cross_entropy(self.y,t)
        return loss
    
    def gradient(self,x,t,learning_rate):
        self.learning_rate = learning_rate
        f = lambda w : self.loss(x,t)
        for i in range(len(self.W)):
            self.W['W'+str(i+1)] -= self.learning_rate*numerical_gradient(f,self.W['W'+str(i+1)])
            self.b['b'+str(i+1)] -= self.learning_rate*numerical_gradient(f,self.b['b'+str(i+1)])
            
        
#     def fit(self,x,t,learning_rate,epochs):
#         for epoch in range(epochs):
#             self.gradient(self,x,t)
        
    
    def accuracy(self,x,t):
        result = self.predict(x)
        acc = sum(np.argmax(result,axis=1) == np.argmax(t,axis=1))/len(t)
        return acc

def numerical_gradient(f,x):
    h = 1e-4
    grad = np.zeros_like(x)
    it = np.nditer(x,flags=['multi_index'],op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = tmp_val + h
        fxh1 = f(x)
        x[idx] = tmp_val - h
        fxh2 = f(x)
        grad[idx] = (fxh1-fxh2)/(2*h)
        x[idx] = tmp_val
        it.iternext()
    return grad

def par_der(f,x):
    h = 1e-4
    grad = np.zeros_like(x)
    it = np.nditer(x,flags=['multi_index'],op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x)
        x[idx] = float(tmp_val) - h
        fxh2 = f(x)
        grad[idx] = (fxh1 - fxh2)/(2*h)
        x[idx] = tmp_val
        it.iternext()
    return grad

def cross_entropy(y,t):
    epsilon = 1e-7
    # y = Activation.softmax(y)
    return -np.sum(t*np.log(y+epsilon))/y.shape[0]

##data
def make_onehot(x):
    col = np.unique(x).size
    row = x.size
    data = np.zeros((row,col))
    for i in np.arange(row):
        data[i,x[i]] = 1
    return data

In [15]:
X = load_iris()['data']
Y = make_onehot(load_iris()['target'])
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='relu', input_shape=(4, )))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# %%time
model.fit(X, Y, epochs=100, batch_size=150)


Epoch 1/100
1/1 [==============================] - 0s 439ms/step - loss: 1.2101 - accuracy: 0.3333
Epoch 2/100
1/1 [==============================] - 0s 6ms/step - loss: 1.1145 - accuracy: 0.3333
Epoch 3/100
1/1 [==============================] - 0s 8ms/step - loss: 1.0559 - accuracy: 0.3333
Epoch 4/100
1/1 [==============================] - 0s 7ms/step - loss: 1.0172 - accuracy: 0.5533
Epoch 5/100
1/1 [==============================] - 0s 22ms/step - loss: 0.9866 - accuracy: 0.5800
Epoch 6/100
1/1 [==============================] - 0s 6ms/step - loss: 0.9612 - accuracy: 0.4867
Epoch 7/100
1/1 [==============================] - 0s 99ms/step - loss: 0.9412 - accuracy: 0.4133
Epoch 8/100
1/1 [==============================] - 0s 171ms/step - loss: 0.9236 - accuracy: 0.4000
Epoch 9/100
1/1 [==============================] - 0s 86ms/step - loss: 0.9072 - accuracy: 0.3733
Epoch 10/100
1/1 [==============================] - 0s 24ms/step - loss: 0.8921 - accuracy: 0.3667
Epoch 11/100
1/1 [===

In [24]:
X = load_iris()['data']
Y = make_onehot(load_iris()['target'])

# %%time
model2 = Network()

epochs = 100
for e in range(epochs):
    e += 1
    model2.gradient(X, Y, 1e-3)
    print(e, model2.loss(X, Y), model2.accuracy(X, Y))


1 10.745397068379418 0.3333333333333333
2 10.745397068379416 0.3333333333333333
3 10.745397068379416 0.3333333333333333
4 10.745397068379416 0.3333333333333333
5 10.745397068379415 0.3333333333333333
6 10.745397068379415 0.3333333333333333
7 10.745397068379413 0.3333333333333333
8 10.745397068379413 0.3333333333333333
9 10.745397068379413 0.3333333333333333
10 10.745397068379411 0.3333333333333333
11 10.745397068379411 0.3333333333333333
12 10.74539706837941 0.3333333333333333
13 10.74539706837941 0.3333333333333333
14 10.74539706837941 0.3333333333333333
15 10.745397068379408 0.3333333333333333
16 10.745397068379408 0.3333333333333333
17 10.745397068379408 0.3333333333333333
18 10.745397068379408 0.3333333333333333
19 10.745397068379408 0.3333333333333333
20 10.745397068379406 0.3333333333333333
21 10.745397068379406 0.3333333333333333
22 10.745397068379404 0.3333333333333333
23 10.745397068379404 0.3333333333333333
24 10.745397068379404 0.3333333333333333
25 10.745397068379402 0.3333

In [32]:
class Multilayer:
    def __init__(self):
        self.x = None
        self.y = None
    def forward(self, X, Y):
        self.x = X
        self.y = Y
        out = self.x * self.y
        return out
    def backward(self, dout):
        dx = dout * self.x
        dy = dout * self.y
        return dx, dy

class Addlayer:
    def __init__(self):
        self.x = None
        self.y = None
    def forward(self, X, Y):
        self.x = X
        self.y = Y
        out = self.x * self.y
        return out
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

class Relu:
    def __init__(self):
        self.mask = None
    def forward(self, X):
        self.mask = (x <= 0)
        out = X.copy()
        out[self.mask] = 0
        return out
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx
        
class Sigmoid:
    def __init__(self):
        self.out = None
    def forward(self, X):
        out = Activation.sigmoid(x)
        self.out = out
        return self.out
    def backward(self, dout):
        dx = dout * self.out * (1. - self.out)
        return dx

class Affine:
    def __init__(self):
        self.w = None
        self.b = None
        self.x = None
        self.origin_shape = None
        self.dw = None
        self.db = None
    def forward(self, x):
        self.origin_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x
        out = np.dot(self.x, self.w) + self.b
        return out
    def backward(self, dout):
        dx = np.dot(dout, self.w.T)
        self.dw = np.dot()
        self.db = np.dot()
        return dx

In [33]:
mul = Multilayer()
mul.forward(100, 2)
mul.backward(100)

add = Addlayer()
add.forward(100, 2)
# add.backward(100)

np.random.seed(100)
x = np.random.randint(-5, 50, 30)
x = x.reshape(5, 6)

200